In [ ]:
from lib.v2.parameter_estimation import ParemeterEstimation

estimator = ParemeterEstimation(path = "data/growth_full2.csv", sep=";", col_temp="Temp", col_uia="NH4", col_do="DO", col_doc="DOC")

# intial setup
estimator.set_data_for_interpolation(path = "data/biochem.csv")
estimator.set_conditional_parameter(cond_temp=(25, 28, 32, 33), cond_uia=(0, 0.0001, 0.06, 1),
                                cond_do=(4, 6, 9, 10), cond_csc=(0, 0, 0.5, 3))
estimator.set_food_availablelity_data()
estimator.set_growth_paremater(w0=0.05, wn=40, n0=100, sr=0.92)
estimator.set_partial_harvest_parameter(doc=[60, 90, 100], ph=[0.1, 0.1, 0.1], final_doc=120)
estimator.set_pond_data(area=1000)

# estimator.fit()

In [ ]:
import numpy as np
T = 70
sr = 0.92
m = -np.log(sr)/T

weight = []
fr = []

for t in range(1, T):
    weight.append(estimator.single_operation(0, t, m, 0.046844136331559816, 0.09628233481397985, 0.09115163561079387, 0.029125989406487265, 0.05, 0.05))
    fr.append(estimator.fr)

# weight = [estimator.single_operation(0, t, m, 0.046844136331559816, 0.09628233481397985, 0.09115163561079387, 0.029125989406487265, 0.05, 0.05) for t in range(T) if t != 0]


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(weight)

In [1]:
import seaborn as sns

In [2]:
df = sns.load_dataset("iris")

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
